In [31]:
import numpy as np
import math
import random

In [74]:
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self.label = label
        self._op = _op
        
    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward

        return out

    def __radd__(self, other):
        return self + other

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def __rmul__(self, other):
        return self * other
    
    def __pow__(self, other):
        assert isinstance(other, (int, float))
        out = Value(self.data ** other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data ** (other - 1)) * out.grad
        out._backward = _backward

        return out
    
    def __truediv__(self, other):
        return self * (other ** -1)
    
    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        return self + (-other)

    def backward(self):
        visited = set()
        topo = []
        def build_topo(node):
            if node not in visited:
                visited.add(node)
                for child in node._prev:
                    build_topo(child)
                topo.append(node)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()
    
    def relu(self):
        out = Value(np.maximum(0, self.data), (self,), 'ReLU')

        def _backward():
            self.grad += (self.data > 0) * out.grad
        out._backward = _backward

        return out
    
    def sigmoid(self):
        out = Value(1 / (1 + np.exp(-self.data)), (self,), 'Sigmoid')

        def _backward():
            self.grad += (1 - out.data) * out.data * out.grad
        out._backward = _backward

        return out

    def tanh(self):
        out = Value(np.tanh(self.data), (self,), 'Tanh')

        def _backward():
            self.grad += (1 - out.data ** 2) * out.grad
        out._backward = _backward

        return out

In [21]:
a = Value(3.0, label='a')
b = Value(2.0, label='b')
c = Value(4.0, label='c')
d = Value(-1.0, label='d')
e = a + b
e.label = 'e'
f = c + d
f.label = 'f'
g = e * f
g.label = 'g'
values = [a, b, c, d, e, f, g]
for v in values:
    print(v._prev)

set()
set()
set()
set()
{Value(data=3.0, grad=0.0, label=a), Value(data=2.0, grad=0.0, label=b)}
{Value(data=-1.0, grad=0.0, label=d), Value(data=4.0, grad=0.0, label=c)}
{Value(data=3.0, grad=0.0, label=f), Value(data=5.0, grad=0.0, label=e)}


In [79]:
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'
o.backward()

In [81]:
values = [x1, x2, w1, w2, b, x1w1, x2w2, x1w1x2w2, n, o]
for v in values:
    print(v)

Value(data=2.0, grad=-1.4999999999999996, label=x1)
Value(data=0.0, grad=0.4999999999999999, label=x2)
Value(data=-3.0, grad=0.9999999999999998, label=w1)
Value(data=1.0, grad=0.0, label=w2)
Value(data=6.881373587019543, grad=0.4999999999999999, label=b)
Value(data=-6.0, grad=0.4999999999999999, label=x1*w1)
Value(data=0.0, grad=0.4999999999999999, label=x2*w2)
Value(data=-6.0, grad=0.4999999999999999, label=x1*w1 + x2*w2)
Value(data=0.8813735870195432, grad=0.4999999999999999, label=n)
Value(data=0.7071067811865476, grad=1.0, label=o)


In [27]:
a = Value(3.0, label='a')
b = Value(4.0, label='b')
c = a + b
c.label = 'c'
d = Value(6.0, label='d')
e = c * d
e.label = 'e'
L1 = e


h = 0.0001
a = Value(3.0, label='a')
b = Value(4.0, label='b')
a += h
c = a + b
c.label = 'c'
d = Value(6.0, label='d')
e = c * d
e.label = 'e'
L2 = e

# dL/da = dL/de * de/dc * dc/da
# dL/de = 1, de/dc = d, dc/da = 1


print((L2.data - L1.data) / h)

5.9999999999860165


In [75]:
class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        activation = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        return activation.tanh()

    def parameters(self):
        return self.w + [self.b] 

In [87]:
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

In [103]:
class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(sz) - 1)]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]
    
    def predict(self, x):
        prediction = self(x)
        return 1 if prediction.data > 0.5 else 0

In [112]:
model = MLP(2, [4, 1])
train_data = [[0, 0], [0, 1], [1, 0], [1, 1]]
train_labels = [0, 1, 1, 0]

In [118]:
num_epochs = 30
learning_rate = 0.1
for epoch in range(num_epochs):
    # forward pass
    ypred = [model(x) for x in train_data]
    loss = sum((yout - ygt) ** 2 for yout, ygt in zip(ypred, train_labels))

    # zero gradients
    for p in model.parameters():
        p.grad = 0.0

    # backward pass
    loss.backward()

    # update parameters
    for p in model.parameters():
        p.data -= learning_rate * p.grad

    print(f'Epoch {epoch}: loss = {loss.data}')

Epoch 0: loss = 4.717320350658328
Epoch 1: loss = 0.9603950030907694
Epoch 2: loss = 0.9442113229632922
Epoch 3: loss = 0.9315654965017548
Epoch 4: loss = 0.9208547526385173
Epoch 5: loss = 0.9114451516748208
Epoch 6: loss = 0.9029227091769282
Epoch 7: loss = 0.8949974624987704
Epoch 8: loss = 0.8874628864323124
Epoch 9: loss = 0.8801696790237641
Epoch 10: loss = 0.8730081305456315
Epoch 11: loss = 0.8658962111993187
Epoch 12: loss = 0.8587714504662954
Epoch 13: loss = 0.8515853347082393
Epoch 14: loss = 0.8442993936465463
Epoch 15: loss = 0.8368824348837318
Epoch 16: loss = 0.8293085709740556
Epoch 17: loss = 0.8215558030075076
Epoch 18: loss = 0.8136050023268492
Epoch 19: loss = 0.8054391829864063
Epoch 20: loss = 0.7970429913660821
Epoch 21: loss = 0.7884023619481013
Epoch 22: loss = 0.7795043034402297
Epoch 23: loss = 0.7703367896151267
Epoch 24: loss = 0.7608887359958869
Epoch 25: loss = 0.7511500478693647
Epoch 26: loss = 0.7411117277207738
Epoch 27: loss = 0.7307660315089435
Epo

In [119]:
print(model.predict([0, 1]))

1
